In [130]:
def to_signed_bin(num, bits=64):
    return format(num & (2**bits - 1), f'0{bits}b')

In [131]:
def divstep(delta, f, g, u, v, r, s):
    g_and_1 = g & 1
    cond = (int(delta > 0)) & (g_and_1 == 1)

    if cond:
        delta = 1 - delta

        f_new = g 
        u_new = r
        v_new = s

        g_new = (g - f) >> 1
        r_new = (r - u) >> 1
        s_new = (s - v) >> 1

    else:
        delta = 1 + delta

        f_new = f
        u_new = u
        v_new = v

        g_new = (g + (g_and_1 * f)) >> 1
        r_new = (r + (g_and_1 * u)) >> 1
        s_new = (s + (g_and_1 * v)) >> 1



    return delta, f_new, g_new, u_new, v_new, r_new, s_new

P = (2**255) - 19
from random import randint
F = randint(0, 2**30 - 1)
G = randint(0, 2**30 - 1)
delta = 0.5
print(F)
print(G)


348931824
1047687559


In [132]:
f = F
g = G
u = -2**30
v = 0
r = 0
s = 2**30
print(f,g)
delta = 0.5
STEPS = 30
for _ in range(STEPS):
    cond_record = (int(delta > 0)) & ((g & 1) == 1)
    delta, f, g, u, v, r, s = divstep(delta, f, g, u, v, r, s)
    #print(f)
    #print(g)

    print(f"\n Iteration {_} ")
    print(f"Condition is {cond_record} \n")

    print(f, u, v)
    fuv30 = (u << 2) + (v << 33) + f
    print((len(bin(abs(f)))-2))

    print(fuv30)
    print(f'fuv = {to_signed_bin(fuv30)}')
    print(f"f   = {to_signed_bin(f)}")
    print(f"u   = {to_signed_bin(u << 2)}")
    print(f"v   = {to_signed_bin(v << 33)}")
    assert((len(bin(fuv30))-2) <= 64)

    print(g, r, s)
    print((len(bin(abs(g)))-2))

    grs30 = ((r << 2) + (s << 33) + g)
    print(grs30)
    print(f'grs = {to_signed_bin(grs30)}')
    print(f"g   = {to_signed_bin(g)}")
    print(f"r   = {to_signed_bin(r << 2)}")
    print(f"s   = {to_signed_bin(s << 33)}")
    assert((len(bin(grs30))-2) <= 64)





348931824 1047687559

 Iteration 0 
Condition is 1 

1047687559 0 1073741824
30
9223372037902463367
fuv = 1000000000000000000000000000000000111110011100100111000110000111
f   = 0000000000000000000000000000000000111110011100100111000110000111
u   = 0000000000000000000000000000000000000000000000000000000000000000
v   = 1000000000000000000000000000000000000000000000000000000000000000
349377867 536870912 536870912
29
4611686020924249419
grs = 0100000000000000000000000000000010010100110100110001010101001011
g   = 0000000000000000000000000000000000010100110100110001010101001011
r   = 0000000000000000000000000000000010000000000000000000000000000000
s   = 0100000000000000000000000000000000000000000000000000000000000000

 Iteration 1 
Condition is 1 

349377867 536870912 536870912
29
4611686020924249419
fuv = 0100000000000000000000000000000010010100110100110001010101001011
f   = 0000000000000000000000000000000000010100110100110001010101001011
u   = 0000000000000000000000000000000010000000000000

In [133]:
print(f, u, v)
fuv30 = (u << 33) + (v << 2) + f
print(fuv30)
print(f'fuv = {to_signed_bin(fuv30)}')
print(f"f   = {to_signed_bin(f)}")
print(f"u   = {to_signed_bin(u << 33)}")
print(f"v   = {to_signed_bin(v << 2)}")
assert((len(bin(fuv30))-2) <= 64)
print(g, r, s)
grs30 = ((r << 33) + (s << 2) + g)
print(grs30)
print(f'grs = {to_signed_bin(grs30)}')
print(f"g   = {to_signed_bin(g)}")
print(f"r   = {to_signed_bin(r << 33)}")
print(f"s   = {to_signed_bin(s << 2)}")
assert((len(bin(grs30))-2) <= 64)

-83311 32388 -74596
278210801184001
fuv = 0000000000000000111111010000011111111111111110100010110100000001
f   = 1111111111111111111111111111111111111111111111101011101010010001
u   = 0000000000000000111111010000100000000000000000000000000000000000
v   = 1111111111111111111111111111111111111111111110110111001001110000
-45248 5015 -44703
43078521754820
grs = 0000000000000000001001110010110111111111111111001001010011000100
g   = 1111111111111111111111111111111111111111111111110100111101000000
r   = 0000000000000000001001110010111000000000000000000000000000000000
s   = 1111111111111111111111111111111111111111111111010100010110000100


In [134]:
import numpy as np

# Construct the matrices
A = np.array([[u, v], [r, s]])
B = np.array([[F, G]])

# Perform matrix multiplication
result = np.dot(A, B.T)

print(result[0]>>30, result[1]>>30)


[-62261] [-41989]


In [136]:
# ------------------------------------------------------------
# 1.  打包／解包工具
# ------------------------------------------------------------
def pack(i: int, f_low: int, u: int, v: int) -> int:
    """將 (f mod 2^i, u, v) 塞進 single limb"""
    return ((f_low & ((1 << i) - 1)) << 33) - (u << (i + 31)) - (v << i)


def unpack(i: int, limb: int):
    """
    從 single-limb 取回 (f mod 2^i, u, v)。
    i==0 時走簡化路徑；其餘用
        ─ 先抓 u   : round(−limb / 2^{i+31})
        ─ 再扣掉 u : limb + 2^{i+31}u = 2^{33}f_low − 2^{i}v
        ─ 用 bit-i 與正負號決定 v ∈ {−1,0,1}
        ─ 扣 v 後除 2^{33} 得 f_low
    """
    if i == 0:
        u = (-limb + (1 << 30)) >> 31        # 最近整數
        v = -(limb + (u << 31))
        return 0, u, v                       # f_low=0  (因為 i=0)
    # --- 1. u ---
    u = (-limb + (1 << (i + 30))) >> (i + 31)
    # --- 2. v ---
    tmp = limb + (u << (i + 31))             # 2^33 f_low − 2^i v
    t   = tmp >> i                           # 2^{33-i}f_low − v
    v = 0 if (t & 1) == 0 else (-1 if t > 0 else 1)
    # --- 3. f_low ---
    f_low = (tmp + (v << i)) >> 33           # 去掉 −2^i v 再縮 33 位
    return f_low, u, v


def unpack_final(limb: int):
    """i==0 時只剩 u,v（最快的解包）"""
    u = (-limb + (1 << 30)) >> 31
    v = -(limb + (u << 31))
    return u, v


# ------------------------------------------------------------
# 2.  單步 divstep 但「僅操縱兩顆 limb」
# ------------------------------------------------------------
def divstep_single(delta, fuv, grs, i):
    """
    • 傳入  : delta, current-limb fuv, grs, 以及 i(=還剩幾輪)
    • 內部  : 先解 (f,u,v,r,s) → 跑經典 divstep → 重新 pack(i-1,…)
    • 回傳  : 更新後的 (delta, fuv', grs', i-1)
    """
    # --- 解包 ---
    f_low, u, v = unpack(i, fuv)
    g_low, r, s = unpack(i, grs)

    g_lsb = g_low & 1
    cond  = (delta > 0) and (g_lsb == 1)

    # --- divstep 核心 ---
    if cond:                      # branch ①
        delta = 1 - delta
        f_low_new, u_new, v_new = g_low,  r,  s
        g_low_new               = (g_low - f_low) >> 1
        r_new, s_new            = (r - u) >> 1, (s - v) >> 1
    else:                        # branch ②
        delta = 1 + delta
        f_low_new, u_new, v_new = f_low,  u,  v
        g_low_new               = (g_low + g_lsb * f_low) >> 1
        r_new, s_new            = (r + g_lsb * u) >> 1, (s + g_lsb * v) >> 1

    # --- 重新打包，用 i' = i-1 ---
    i_next  = i - 1
    fuv_new = pack(i_next, f_low_new, u_new, v_new)
    grs_new = pack(i_next, g_low_new, r_new, s_new)
    return delta, fuv_new, grs_new, i_next


# ------------------------------------------------------------
# 3.  Demo -- 30 步後一次解包驗證
# ------------------------------------------------------------
if __name__ == "__main__":
    # (一) 先把原始六個量打進兩顆 limb
    F, G = 780_328_917, 668_388_218
    delta = 0.5
    i     = 30
    fuv   = pack(i, F & ((1 << i) - 1),  -2**30, 0)      #  (f,u,v)
    grs   = pack(i, G & ((1 << i) - 1),   0,     -2**30) #  (g,r,s)

    # (二) 跑滿 30 輪
    for _ in range(30):
        delta, fuv, grs, i = divstep_single(delta, fuv, grs, i)

    # (三) 最後 i==0，再拆出 u,v,r,s
    u_fin, v_fin = unpack_final(fuv)
    r_fin, s_fin = unpack_final(grs)

    print("delta =", delta)
    print("u, v  =", u_fin, v_fin)
    print("r, s  =", r_fin, s_fin)


delta = 1.5
u, v  = -29253 0
r, s  = 11950 0
